# HuggingFace Supervised Fine-tuning Trainer (SFT)

https://huggingface.co/docs/trl/en/sft_trainer

## TinyLlamma
https://arxiv.org/pdf/2401.02385
https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v0.1
https://huggingface.co/facebook/opt-350m
https://huggingface.co/facebook/MobileLLM-125M

## Example scripts
https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py

## Inspired by
https://colab.research.google.com/github/huggingface/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb


In [1]:
# ! pip install wandb

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

In [ ]:
# Select the base model
model_name = "HuggingFaceTB/SmolLM2-135M"

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.1"
model_name = "facebook/opt-350m"

model_name = "facebook/MobileLLM-125M"

os.environ["WANDB_PROJECT"] = "tiny-llama-ft"
os.environ["WANDB_DIR"] = "./temp"
os.environ["WANDB_JOB_NAME"] = "some-job-name"

## 1. Load the model to appropriate available device (CPU/GPU)

In [3]:
# Check the machine in use and set the device to use for training
# cuda = GPU, mps = Metal Performance Shaders on macOS or Apple GPU, cpu otherwise
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Print device info
print("Model loaded to: ", device)



# Load the pretrained model & move it to the specified device
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Setup for the model specific chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

Model loaded to:  cpu


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


## 2. Prepare the dataset

**Dataset format support**

https://huggingface.co/docs/trl/en/sft_trainer#dataset-format-support

In [4]:
# Load a sample dataset
from datasets import load_dataset

dataset_name = "HuggingFaceTB/smoltalk"
dataset_split = "everyday-conversations"

ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")



## 3. Setup the training configuration

**SFTConfig**

https://huggingface.co/docs/trl/v0.12.2/en/sft_trainer#trl.SFTConfig

This object specifies hyperparameters and settings for the fine-tuning process. It’s tailored to supervised fine-tuning tasks, often used for adapting language models to specific tasks or datasets.

In [5]:
from datetime import datetime

# Get the current timestamp
current_time = datetime.now()

# Create a readable timestamp
formatted_time = current_time.strftime("%b-%d-%Y-%H-%M-%S")

# Create a name for the run
wandb_run_name = f"FT_run_{formatted_time}"

# Adjust the model
fine_tuned_model_name = f"fine-tuned-chat-model"

# Model assets output folder
model_output_folder = "c:/temp/sft_output"

# SFTrainer configuration
sft_config = SFTConfig(

    # Output directory for model assets
    output_dir = model_output_folder,  

    # Hyperparameter : Controls maximum number of steps to be executed
    # Maximum number of gradient update steps during training.
    max_steps=100,  

    # Common starting point for fine-tuning
    # The initial learning rate for the optimizer.
    learning_rate=5e-5,  

    # Set according to your GPU memory capacity
    # Number of training samples per device in each batch. Smaller values help fit large models into memory-constrained GPUs.
    per_device_train_batch_size=4,  

    # Frequency of logging training metrics
    # Logs metrics (e.g., loss) every 10 steps during training.
    logging_steps=10,  

    # Frequency of saving model checkpoints
    # Saves model checkpoints every 100 steps. In case of failure, loss or work will be limited to a maximum of 100 steps
    save_steps=100,  

    # Evaluate the model at regular intervals
    eval_strategy="steps",  

    # Frequency of evaluation
    # Run the model evaluation after every 50 steps
    eval_steps=50,  

    # Use MPS for mixed precision training
    use_mps_device=(
        True if device == "mps" else False
    ),  

    # Set a unique name for your model - used for HuggingFace hub
    hub_model_id=fine_tuned_model_name,  

    # Reporting
    report_to = "wandb",
    run_name = wandb_run_name,
)



## SFTrainer

https://huggingface.co/docs/trl/v0.12.2/en/sft_trainer#trl.SFTTrainer

In [6]:
# Initialize the SFTTrainer
trainer = SFTTrainer(

    # The language model being fine-tuned.
    model=model,

    # Passes the fine-tuning configuration defined above 
    args=sft_config,

    # Training dataset
    train_dataset=ds["train"],

    # Evaluation dataset
    eval_dataset=ds["test"],

    # Tokenizer used
    tokenizer=tokenizer,
    
)

C:\Users\raj\anaconda3\envs\gen-ai-app-dev-course\Lib\site-packages\trl\trainer\sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Train the model

Wandb - configuration
https://docs.wandb.ai/guides/track/environment-variables/

import os
os.environ["WANDB_DISABLED"] = "True"

In [ ]:
import os 

# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{fine_tuned_model_name}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING Path ./temp\wandb\ wasn't writable, using system temp directory
wandb: Currently logged in as: acloudfan (raj-acloudfan). Use `wandb login --relogin` to force relogin
wandb: WARNING Path ./temp\wandb\ wasn't writable, using system temp directory.
